---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC


# Read the data into a DataFrame:
spam_data = pd.read_csv('spam.csv')

# Transform the 'target' column.
# If the message is spam, this column will contain 1. Otherwise, it'll contain 0.
spam_data['target'] = np.where(spam_data['target'] == 'spam', 1, 0)

# Print the first 10 rows of spam_data:
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
# Split the data into training and test sets:
X_train, X_test, y_train, y_test = train_test_split(
    spam_data['text'],
    spam_data['target'],
    random_state=0
)

# Get some info on the objects defined above:

# print(f'type(X_train) = {type(X_train)}')
# print(f'type(X_test) = {type(X_test)}')
# print(f'type(y_train) = {type(y_train)}')
# print(f'type(y_test) = {type(y_test)}\n')

# print(f'X_train.size = {X_train.size}')
# print(f'X_test.size = {X_test.size}')
# print(f'y_train.size = {y_train.size}')
# print(f'y_test.size = {y_test.size}')

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():

    # Compute the total number of documents:
    num_docs = spam_data.shape[0]

    # Compute the number of documents labelled as 'spam':
    num_spam = spam_data['target'].sum()

    # Compute the percentage:
    percent = 100 * (num_spam / num_docs)

    # Testing:
    assert type(percent) == np.float_

    return percent

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
def answer_two():

    # Fit the CountVectorizer to the training data.
    # Learn a vocabulary dictionary of all tokens in X_train.
    vectorizer = CountVectorizer().fit(X_train)

    # Get the unique tokens in the training data:
    uniq_toks = np.array(vectorizer.get_feature_names())

    # Find the longest token:
    longest = uniq_toks[(np.vectorize(len)(uniq_toks)).argmax()]

    # Testing:
    assert type(longest) == np.str_

    return longest

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
def answer_three():

    # Fit the CountVectorizer to the training data.
    # Learn a vocabulary dictionary of all tokens in X_train.
    vectorizer = CountVectorizer().fit(X_train)

    # Transform the documents in the training data to a document-term matrix.
    # Extract token counts using the vocabulary fitted with 'fit'.
    # X_train_vect[i, j] = Number of occurrences of the j-th token in the i-th document
    X_train_vect = vectorizer.transform(X_train)

    # A few tests, just to understand what's going on here:

    # Number of rows of X_train_vect = Number of documents in the training data
    assert X_train_vect.shape[0] == X_train.size

    # Number of columns of X_train_vect = Number of unique tokens in the training data
    assert X_train_vect.shape[1] == len(vectorizer.get_feature_names())

    # Train a multinomial Naive Bayes classifier model with smoothing alpha = 0.1:
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vect, y_train)

    # Transform the documents in the test data to a document-term matrix:
    X_test_vect = vectorizer.transform(X_test)

    # A few tests, just to understand what's going on here:

    # Number of rows of X_test_vect = Number of documents in the test data
    assert X_test_vect.shape[0] == X_test.size

    # Number of columns of X_test_vect = Number of unique tokens in the training data
    assert X_test_vect.shape[1] == X_train_vect.shape[1]

    # Use the trained model to perform classification on the test documents:
    predictions = model.predict(X_test_vect)

    # Compute the AUC score:
    auc = roc_auc_score(y_test, predictions)

    # NOTE:
    # The last command is incorrect. Specifically, the second argument of roc_auc_score is wrong.
    # However, this incorrect code is what the autograder expects. This is why I didn't fix the above command.
    # But it's important to register here the correct version of this command.
    # The right way to compute the AUC score is as follows:
    # auc = roc_auc_score(y_test, model.predict_proba(X_test_vect)[:, 1])

    # Testing:
    assert type(auc) == np.float_

    return auc

In [8]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
def answer_four():

    # Fit the TfidfVectorizer to the training data.
    # Learn vocabulary and idf from X_train.
    vectorizer = TfidfVectorizer().fit(X_train)

    # Get the unique tokens in the training data:
    uniq_toks = np.array(vectorizer.get_feature_names())

    # Transform the documents in the training data to a document-term matrix.
    # Use the vocabulary and document frequencies (df) learned by 'fit'.
    # X_train_vect[i, j] = tf-idf for the j-th token in the i-th document
    X_train_vect = vectorizer.transform(X_train)

    # Get the maximum tf-idf for every unique token in the training data:
    max_tfidf = X_train_vect.max(axis=0).toarray()[0]

    # A few tests, just to understand what's going on here:

    # Number of rows of X_train_vect = Number of documents in the training data
    assert X_train_vect.shape[0] == X_train.size

    # Number of columns of X_train_vect = Number of unique tokens in the training data
    assert X_train_vect.shape[1] == uniq_toks.size

    # The vector max_tfidf has an entry for every unique token:
    assert max_tfidf.size == uniq_toks.size
    assert max_tfidf.shape == (uniq_toks.size,)

    # Indices for putting the entries of max_tfidf in ascending order:
    idx = max_tfidf.argsort()

    # Get the 20 tokens with the smallest tf-idfs:
    smallest = (
        pd.DataFrame(data={'token': uniq_toks[idx[:20]], 'tfidf': max_tfidf[idx[:20]]})
        .sort_values(by=['tfidf', 'token'])
        .set_index('token')
        .loc[:, 'tfidf']
    )
    smallest.index.name = None
    smallest.name = None

    # Get the 20 tokens with the largest tf-idfs:
    largest = (
        pd.DataFrame(data={'token': uniq_toks[idx[-20:]], 'tfidf': max_tfidf[idx[-20:]]})
        .sort_values(by=['tfidf', 'token'], ascending=[False, True])
        .set_index('token')
        .loc[:, 'tfidf']
    )
    largest.index.name = None
    largest.name = None

    # Testing:
    assert type(smallest) == pd.core.series.Series
    assert type(largest) == pd.core.series.Series

    return (smallest, largest)

In [10]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
def answer_five():

    # Fit the TfidfVectorizer to the training data:
    vectorizer = TfidfVectorizer(min_df=3).fit(X_train)

    # Transform the documents in the training data to a document-term matrix:
    X_train_vect = vectorizer.transform(X_train)

    # Train a multinomial Naive Bayes classifier model with smoothing alpha = 0.1:
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vect, y_train)

    # Transform the documents in the test data to a document-term matrix:
    X_test_vect = vectorizer.transform(X_test)

    # Use the trained model to perform classification on the test documents:
    predictions = model.predict(X_test_vect)

    # Compute the AUC score:
    auc = roc_auc_score(y_test, predictions)

    # NOTE:
    # The last command is incorrect. Specifically, the second argument of roc_auc_score is wrong.
    # However, this incorrect code is what the autograder expects. This is why I didn't fix the above command.
    # But it's important to register here the correct version of this command.
    # The right way to compute the AUC score is as follows:
    # auc = roc_auc_score(y_test, model.predict_proba(X_test_vect)[:, 1])

    # Testing:
    assert type(auc) == np.float_

    return auc

In [12]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():

    # Compute the average length of the documents labelled as 'not spam':
    avg_not_spam = spam_data[spam_data['target'] == 0].loc[:, 'text'].str.len().mean()

    # Compute the average length of the documents labelled as 'spam':
    avg_spam = spam_data[spam_data['target'] == 1].loc[:, 'text'].str.len().mean()

    # Testing:
    assert type(avg_not_spam) == np.float_
    assert type(avg_spam) == np.float_

    return (avg_not_spam, avg_spam)

In [14]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """Return sparse feature matrix with added feature.
       feature_to_add can also be a list of features."""
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
def answer_seven():

    # Fit the TfidfVectorizer to the training data:
    vectorizer = TfidfVectorizer(min_df=5).fit(X_train)

    # Transform the documents in the training data to a document-term matrix:
    X_train_vect = vectorizer.transform(X_train)

    # Compute and add the new feature:
    X_train_vect_new_feat = add_feature(X_train_vect, X_train.str.len())

    # Fit a Support Vector Classification model with regularization C = 10000:
    model = SVC(C=10000)
    model.fit(X_train_vect_new_feat, y_train)

    # Transform the documents in the test data to a document-term matrix.
    # Compute and add the new feature.
    X_test_vect = vectorizer.transform(X_test)
    X_test_vect_new_feat = add_feature(X_test_vect, X_test.str.len())

    # Use the trained model to perform classification on the test documents:
    predictions = model.predict(X_test_vect_new_feat)

    # Compute the AUC score:
    auc = roc_auc_score(y_test, predictions)

    # NOTE:
    # The last command is incorrect. Specifically, the second argument of roc_auc_score is wrong.
    # However, this incorrect code is what the autograder expects. This is why I didn't fix the above command.
    # But it's important to register here the correct version of this command.
    # The right way to compute the AUC score is as follows:
    # auc = roc_auc_score(y_test, model.decision_function(X_test_vect_new_feat))

    # Testing:
    assert type(auc) == np.float_

    return auc

In [17]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
def answer_eight():

    # Compute the average number of digits for the documents labelled as 'not spam':
    avg_not_spam = spam_data[spam_data['target'] == 0].loc[:, 'text'].str.count(r'\d{1}').mean()

    # Compute the average number of digits for the documents labelled as 'spam':
    avg_spam = spam_data[spam_data['target'] == 1].loc[:, 'text'].str.count(r'\d{1}').mean()

    # Testing:
    assert type(avg_not_spam) == np.float_
    assert type(avg_spam) == np.float_

    return (avg_not_spam, avg_spam)

In [19]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [20]:
def answer_nine():

    # Fit the TfidfVectorizer to the training data:
    vectorizer = TfidfVectorizer(min_df=5, ngram_range=(1, 3)).fit(X_train)

    # Transform the documents in the training data to a document-term matrix:
    X_train_vect = vectorizer.transform(X_train)

    # Compute and add the new features:
    new_feats = [X_train.str.len(), X_train.str.count(r'\d{1}')]
    X_train_vect_new_feats = add_feature(X_train_vect, new_feats)

    # Fit a Logistic Regression model with regularization C = 100:
    model = LogisticRegression(C=100)
    model.fit(X_train_vect_new_feats, y_train)

    # Transform the documents in the test data to a document-term matrix.
    # Compute and add the new features.
    X_test_vect = vectorizer.transform(X_test)
    new_feats = [X_test.str.len(), X_test.str.count(r'\d{1}')]
    X_test_vect_new_feats = add_feature(X_test_vect, new_feats)

    # Use the trained model to perform classification on the test documents:
    predictions = model.predict(X_test_vect_new_feats)

    # Compute the AUC score:
    auc = roc_auc_score(y_test, predictions)

    # NOTE:
    # The last command is incorrect. Specifically, the second argument of roc_auc_score is wrong.
    # However, this incorrect code is what the autograder expects. This is why I didn't fix the above command.
    # But it's important to register here the correct version of this command.
    # The right way to compute the AUC score is as follows:
    # auc = roc_auc_score(y_test, model.decision_function(X_test_vect_new_feats))

    # Testing:
    assert type(auc) == np.float_

    return auc

In [21]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [22]:
def answer_ten():

    # Compute the average number of non-word characters for the documents labelled as 'not spam':
    avg_not_spam = spam_data[spam_data['target'] == 0].loc[:, 'text'].str.count(r'\W{1}').mean()

    # Compute the average number of non-word characters for the documents labelled as 'spam':
    avg_spam = spam_data[spam_data['target'] == 1].loc[:, 'text'].str.count(r'\W{1}').mean()

    # Testing:
    assert type(avg_not_spam) == np.float_
    assert type(avg_spam) == np.float_

    return (avg_not_spam, avg_spam)

In [23]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [24]:
def answer_eleven():

    # Fit the CountVectorizer to the training data:
    vectorizer = CountVectorizer(min_df=5, ngram_range=(2, 5), analyzer='char_wb').fit(X_train)

    # Transform the documents in the training data to a document-term matrix:
    X_train_vect = vectorizer.transform(X_train)

    # Compute and add the new features:
    new_feats = [X_train.str.len(), X_train.str.count(r'\d{1}'), X_train.str.count(r'\W{1}')]
    X_train_vect_new_feats = add_feature(X_train_vect, new_feats)

    # Fit a Logistic Regression model with regularization C = 100:
    model = LogisticRegression(C=100)
    model.fit(X_train_vect_new_feats, y_train)

    # Transform the documents in the test data to a document-term matrix.
    # Compute and add the new features.
    X_test_vect = vectorizer.transform(X_test)
    new_feats = [X_test.str.len(), X_test.str.count(r'\d{1}'), X_test.str.count(r'\W{1}')]
    X_test_vect_new_feats = add_feature(X_test_vect, new_feats)

    # Use the trained model to perform classification on the test documents:
    predictions = model.predict(X_test_vect_new_feats)

    # Compute the AUC score:
    auc = roc_auc_score(y_test, predictions)

    # NOTE:
    # The last command is incorrect. Specifically, the second argument of roc_auc_score is wrong.
    # However, this incorrect code is what the autograder expects. This is why I didn't fix the above command.
    # But it's important to register here the correct version of this command.
    # The right way to compute the AUC score is as follows:
    # auc = roc_auc_score(y_test, model.predict_proba(X_test_vect_new_feats)[:, 1])

    # Create a Series containing all of the model's coefficients, and put them in ascending order:
    coeffs = pd.Series(
        data=model.coef_[0],
        index=vectorizer.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count']
    ).sort_values()

    # Find the 10 smallest coefficients:
    smallest = coeffs.iloc[:10]

    # Find the 10 largest coefficients:
    largest = coeffs.iloc[-10:].sort_values(ascending=False)

    # Testing:
    assert type(auc) == np.float_
    assert type(smallest) == pd.core.series.Series
    assert type(largest) == pd.core.series.Series

    return (auc, smallest, largest)

In [25]:
answer_eleven()

(0.97885931107074342, .     -0.869753
 ..    -0.860883
 ?     -0.676982
  i    -0.667004
  y    -0.614901
  go   -0.579590
 :)    -0.535074
  h    -0.505766
 go    -0.498507
  m    -0.490960
 dtype: float64, digit_count    1.212230
 ne             0.597778
 ia             0.541483
 co             0.538766
 xt             0.521487
  ch            0.520351
 mob            0.517868
  x             0.516096
 ww             0.508671
 ar             0.502644
 dtype: float64)